Свойкин Е.В. 2021 г.

### Вычисление продуктивности горизонтальной скважины по корреляциям Joshi и Ozkan

Безразмерный коэффициент продуктивности горизонатльной скважины на установившемся режиме рассчитывается следующим образом:

$$ J_d = \frac{1}{(Log(\frac{r_e}{r_w}) + S_h)} $$

Где Ozkan предложил следующие коэффициенты:

$$ S_h = -log(\frac{L}{4r_w}) - \frac{hα^{-0.5}}{L} Log(4sin(\frac{π(z_w + \frac{r_w}{2α^{-0.5}})}{h})) sin(\frac{πr_w}{2α^{-0.5}h}) $$

Joshi в свою очередь:

$$ S_h = -Log(\frac{r_e}{r_w}) + Log(\frac{2a + (4a^{2} - L^{2})^{0.5}}{L}) + \frac{α^{-0.5}h}{L} \frac{log(α^{-0.5}(h^{2}+4(|\frac{h}{2} - z_w|)^{2})}{\frac{(α^{-0.5} + 1)}{h r_w}} $$

$$ a = \frac{L(1 + (1 + 64(\frac{r_e}{L})^{4})^{0.5})^{0.5}}{2\sqrt{2}} $$

где: $ α = \frac{k_v}{k_h} $

Псевдорадиальный скин-фактор:

$$ S_{Jdss} = \frac{1}{J_d} - Log(\frac{r_e}{r_w}) $$

Дебит на установившемся режиме:

$$ Q_{Jd} = \frac{kh}{18.41mu}\frac{dP}{B} J_d $$

ref "Ozkan (SPE 16378)"

ref "Joshi (Horizontal Well Technology book) and it's Economides modification to anisotropic reservoir (SPE 20717)"

In [2]:
from model_create import ModelGenerator, clear_folders
clear_folders()
# Параметры, которые необходимо расчитать:
parameters = ['FOPT', 'WOPR:*', 'FPR', 'WBHP:P1',]# 'FWPT', 'FLPT', 'FGPT', 'FWIT']

# Зададим дату начала расчета и продолжительность:
start_date = u"1 'JAN' 2020"
mounths = 5 # количество итераций;
days = 60 # продолжительность итерации;

# Зададим размеры модели:
lgr = True # локальное измельчение сетки по центру;
cells_cy = 20 # количество измельченных ячеек по y;
cells_cx = 4 # количество измельченных ячеек по x;
cells_v = 10 # размер измельченных ячеек, м;
nx = 42 # количество ячеек по х;
ny = 42 # количество ячеек по y;
nz = 10 # количество ячеек по z;
dx = 35.7
dy = 35.7
lx = 1500 # длина модели по x, м;
ly = lx # длина модели по y, м;
dz = 1 # размер ячейки по z, м;
tops_depth = 2500 # глубина залегания пласта, м;

# Зададим пористость и проницаемость:
por = 0.3 # пористость, д.ед;

# Зададим название, расположение добываюещей скважины и ее режим работы:
horizontal = [True] # Идентификатор горизонтальной скважины для скрипта
prod_names = ['P1'] # название скважин;
prod_xs=[20] # координата скважин по x;
prod_ys=[12] # координата скважин по y (если horizontal=true - начальная координата);
y_stop = [32] # координата конца горизонтальной скважины, если horizontal=true;
prod_z1s=[1] # начало интервала вскрытия по z;
prod_z2s=[6] # конец интервала вскрытия по z (если horizontal=true, то показывает уровень вскрытия);
rezim = ['BHP'] # Режим работы скважины
prod_q_oil = ['1*'] # дебит, м3/сут;
prod_bhp = ['200'] # забойное давление, атм;

# Другие настройки модели:
only_prod = True # Модель только с добывающими скважинами
upr_rezim_water = False # Моделируем упруго-водонапорный режим
upr_rezim_gas = False # Моделируем газонапорный режим
neogr_plast = True # Моделируем неограниченный пласт

# Задаем скин, радиус скважины (последовательно для добывающих и нагнетательных скважин):
skin = [0]
rw = [0.078]

# Свойства продукции:
oil_den = 896 # плотность нефти, кг/м3;
wat_den = 1190 # плотность воды, кг/м3;
gas_den = 0.9 # плотность газа, кг/м3;

# Свойства поороды:
rock_compr = 1.5E-005 # сжимаемость породы, Па^-1

# EQUILIBRIUM DATA:
p_depth = 2500 # Глубина замера пластового давления, м;
p_init = 286  # Начальное пластовое давление, атм;
o_w_contact = 2550 # Глубина ВНК, м;
pc_woc = 0 # Капиллярное давление на ВНК 
g_o_contact = 2400 # Глубина ГНК, м;
pc_goc = 0 # Капиллярное давление на ГНК

In [3]:
#выбираем шаблон data файла для различных симуляторов
template = 2 # : 1-opm; 2-ecl (в разработке)
permx_list = [5, 50, 200] # проницаемость по x, мД;
kk = [0.25, 0.5, 1, 3]
model_list = []
for j in range(1, 2):
    for i in range(0, 1):
        permx = permx_list[i]
        permy = permx_list[i] # проницаемость по y, мД;
        permz = permx_list[i]*kk[j] # проницаемость по z, мД;
        model_name = f'TEST_MODEL_HORIZONTAL.{j}{i}'
        result_name = model_name
        model_list.append(' горизонтальная проницаемость - ' + str(permx) + ' мД; анизотропия - ' + str(kk[j]))
        model = ModelGenerator(start_date=start_date, mounths=mounths,
                        days=days, nx=nx, ny=ny, nz=nz, dz=dz, dx=dx, dy=dy, por=por, permx=permx,
                        permy=permy, permz=permz, prod_names=prod_names, prod_xs=prod_xs,
                        prod_ys=prod_ys, prod_z1s=prod_z1s, prod_z2s=prod_z2s, prod_q_oil=prod_q_oil,
                        skin=skin, oil_den=oil_den, wat_den=wat_den, gas_den=gas_den, p_depth=p_depth, 
                        p_init=p_init, o_w_contact=o_w_contact, pc_woc=pc_woc, g_o_contact=g_o_contact,
                        pc_goc=pc_goc, tops_depth = tops_depth, rock_compr=rock_compr,
                        rezim=rezim, prod_bhp=prod_bhp, horizontal=horizontal, y_stop=y_stop, only_prod=only_prod,
                        upr_rezim_water=upr_rezim_water, upr_rezim_gas=upr_rezim_gas, rw=rw, template=template, neogr=neogr_plast)
        model.create_model(model_name, result_name, parameters)

Локальное измельчение в Eclipse можно задать двумя ключевыми словами:
 * `` AUTOREF
 3 3 1 / в RUNSEPEC`` в RUNSPEC
 
 
 * ``AUTOCOAR
-- I1 I2 J1 J2 K1 K2 NX NY NY
 1 15 1 42 1 10 15 42 10 /
 25 42 1 42 1 10 18 42 10 /
/ `` в GRID
 

In [3]:
import math as m

def Jd_horizontal(rw, h, alpha, L, zw, re, Flag):
    'Вычисление продуктивности горизонтальной скважины и псевдоскин-фактора'

    if Flag == 1:   
        'выбор корреляции для расчета Ozkan (SPE 16378) '
        Sh = -m.log(L / 4 / rw) - h * alpha ** (-0.5) / L * m.log(4 * m.sin(3.14 / h * (zw + rw / 2 / alpha ** (-0.5))) * m.sin(3.14 * rw / 2 / alpha ** (-0.5) / h))
    elif Flag == 2:         
        'Joshi (Horizontal Well Technology book) '
        'and it"s Economides modification to anisotropic reservoir (SPE 20717)'
        a = L / 2 / 2 ** 0.5 * (1 + (1 + 64 * (re / L) ** 4) ** 0.5) ** 0.5
        Sh = -m.log(re / rw) + m.log((2 * a + (4 * a ** 2 - L ** 2) ** 0.5) / L) + alpha ** (-0.5) * h / L * m.log(alpha ** (-0.5) * (h ** 2 + 4 * (abs(h / 2 - zw)) ** 2) / (alpha ** (-0.5) + 1) / h / rw)
#     elif Flag == 3:
#        ' Borisov (1964)'
#         '    Sh = -m.log(L / 4 / rw)
#         '     + h * alpha ** (-0.5) / L / m.log(h * alpha ** (-0.5) / 3.14 / rw / (alpha ** (-0.5) + 1))

    Jd = 1 / (m.log(re / rw) + Sh)
    return Jd


def Q_from_Jd(Jd, k, h, mu, B, dp):

    Q = k * h * dp / 18.41 / mu / B * Jd
    return Q


def S_from_Jdss(Jd, rw, re):

    S = 1 / Jd - m.log(re / rw)
    return S


In [4]:
# Расчет продуктивности горизонатльной скважины на установившемся режиме
# Horizontal Well Productivity calculation (Ozkan model, 1989 and Joshi model, 1988)
# надо найти литературу
#model = 1 # 1-Ozkan; 2-Joshi;
def HWP(kk, h, re, rw, L, zw, model, mu, B, dP):
#     if model == 1:
#         print('Lmin: ' + str(round(8.6*kk**(-0.5)*h,1)))
#         print('Lmax: ' + str(round(2*re,1)))
#     elif model == 2:
#         print('Lmin: ' + str(round(kk**(-0.5)*h,1)))
#         print('Lmax: ' + str(round(1.8*re,1)))

    Jd = round(Jd_horizontal(rw, h, kk, L, zw, re, model),2)
    Sh = round(S_from_Jdss(Jd, rw, re),2)
    Qh = Q_from_Jd(Jd, kh, h, mu, B, dP)
    return Qh

In [5]:
# Вариант №2
# Батлер - приток на псевдоустановившемся режиме к протяженной горизонтальной скважине 147 с.
def bat_2(lx, dP, permx, mu, permz, rw, h, L, B):
    pi = 3.14
    Rк = lx/2 # м, радиус контура питания
    dP = dP/10*10**6 # Па, перепад давления
    kh = permx/10**15 # м2, проницаемость (в числовой модели изотропный пласт)
    mu = mu/1000 # Па*с, вязкость
    kv = permz/10**15
    Rm = 0.5*rw*(1+m.sqrt(kh/kv))
    hm = h*m.sqrt(kh/kv)
    Q5 = (6*kh*L*h*dP)/(mu*(lx/2+3*hm/3.14*m.log1p(hm/2/3.14/Rm)))*86400/B*2
    return Q5

In [6]:
def bat_1(lx, dP, permx, mu, permz, rw, h, L, B):
    pi = 3.14
    Rк = lx/2 # м, радиус контура питания
    dP = dP/10*10**6 # Па, перепад давления
    kh = permx/10**15 # м2, проницаемость (в числовой модели изотропный пласт)
    mu = mu/1000 # Па*с, вязкость
    a = lx # м, сторона модели
    kv = permz/10**15
    Rm = 0.5*rw*(1+m.sqrt(kh/kv))
    hm = h*m.sqrt(kh/kv) # lx
    Q5 = (6*kh*L*h*dP)/(mu*lx/2)*86400/B*2
    return Q5

In [7]:
def bat_3(S, lx, ly, L, permx, permz, rw, h, dP, mu, B):
    A = lx*ly
    prow = A/(L**2) 
    if prow < 35:
        P = m.log1p(1+prow)/12
    else:
        P = (m.log1p(prow)+0.1515)/(4*3.14)
    kh = permx/10**15 # м2, проницаемость (в числовой модели изотропный пласт)
    kv = permz/10**15
    mu = mu/1000 # Па*с, вязкость
    dP = dP/10*10**6 # Па, перепад давления
    hm = h*m.sqrt(kh/kv)
    Rm = 0.5*rw*(1+m.sqrt(kh/kv))
    # учет вертикального схождения
    #P_ = 0
    P_ = hm/(2*3.14*L)*(m.log1p(hm/(2*3.14*Rm*m.sin(3.14/2)))+S)
    Q1 = kh*h*dP/mu/(P_+P)*86400/B
    return Q1

In [35]:
# метод для добавления данных на график
import pandas as pd 
import plotly.graph_objects as go
from plotly.offline import iplot
from ecl2df import pvt, EclFiles
import numpy as np
import scipy.interpolate

def figure_plot(rw, kh, h, kk, L, zw, re, lx, ly, S, start_date, title):
    # Получаем забойное и пластовое давление на все иттерации
    start_date = pd.to_datetime(start_date)
    fig = go.Figure()
    trace_name = ['Ozkan', 'Joshi', 'протяж. горизонт. скв.', 'конеч. вертик. трещина', 'вертикальная трещина']
    for k in range(0, 1):
        filename = f'TEST_MODEL_HORIZONTAL.{k}0'
        df_time = pd.read_csv(f'csv_folder/{filename}.csv', parse_dates=[0], index_col=[0], usecols=['Time' ,'FPR', 'WBHP:P1', 'WOPR:P1'])
        y_list1, y_list2, y_list3, y_list4, y_list5 = [], [], [], [], []
        df_y = []
        fig.add_trace(go.Scatter(
                    x=df_time.index,
                    y=df_time['WOPR:P1'],
                    mode='lines',
                    name='OPM; аниз. - '+str(kk[k])))
        df_vol, df_vis, df_timest = [], [], []
        for i in df_time.index:
            df_timest.append(i)
        for i in df_time.values:
            dP = i[1] - i[2]
            eclfiles = EclFiles(f'model_folder/{filename}')
            dframe = pvt.df(eclfiles)
            indic = dframe['KEYWORD'] == 'PVDO'
            res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
            x = res_df['PRESSURE'].values
            y1 = res_df['VOLUMEFACTOR'].values
            z = np.poly1d(np.polyfit(x, y1, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y11 = interp1(new_x)
            new_y21 = interp2(new_x)
            idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
            y_sol = round(new_y11[idx1[0]][0].astype(float),2)
            x_sol = round(new_x[idx1[0]][0].astype(float),2)
            y2 = res_df['VISCOSITY'].values
            z = np.poly1d(np.polyfit(x, y2, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y12 = interp1(new_x)
            new_y22 = interp2(new_x)
            idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
            y_sol = round(new_y12[idx2[0]][0].astype(float),2)
            x_sol = round(new_x[idx2[0]][0].astype(float),2)
            y_list1.append(HWP(kk[k], h, re, rw, L, zw, 1, new_y12[idx2[0]], new_y11[idx1[0]], dP)[0])
            #print('dP : ' + str(dP))
            df_vol.append(new_y11[idx1[0]][0])
            df_vis.append(new_y12[idx2[0]][0])
        df_y.append(df_timest)
        df_y.append(list(df_time['WOPR:P1']))
        df_y.append(df_vol)
        df_y.append(df_vis)
        df_y.append(y_list1)
        for i in df_time.values:
            dP = i[1] - i[2]
            eclfiles = EclFiles(f'model_folder/{filename}')
            dframe = pvt.df(eclfiles)
            indic = dframe['KEYWORD'] == 'PVDO'
            res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
            x = res_df['PRESSURE'].values
            y1 = res_df['VOLUMEFACTOR'].values
            z = np.poly1d(np.polyfit(x, y1, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y11 = interp1(new_x)
            new_y21 = interp2(new_x)
            idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
            y_sol = round(new_y11[idx1[0]][0].astype(float),2)
            x_sol = round(new_x[idx1[0]][0].astype(float),2)
            y2 = res_df['VISCOSITY'].values
            z = np.poly1d(np.polyfit(x, y2, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y12 = interp1(new_x)
            new_y22 = interp2(new_x)
            idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
            y_sol = round(new_y12[idx2[0]][0].astype(float),2)
            x_sol = round(new_x[idx2[0]][0].astype(float),2)
            y_list2.append(HWP(kk[k], h, re, rw, L, zw, 2, new_y12[idx2[0]], new_y11[idx1[0]], dP)[0])
        df_y.append(y_list2)
        for i in df_time.values:
            dP = i[1] - i[2]
            eclfiles = EclFiles(f'model_folder/{filename}')
            dframe = pvt.df(eclfiles)
            indic = dframe['KEYWORD'] == 'PVDO'
            res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
            x = res_df['PRESSURE'].values
            y1 = res_df['VOLUMEFACTOR'].values
            z = np.poly1d(np.polyfit(x, y1, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y11 = interp1(new_x)
            new_y21 = interp2(new_x)
            idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
            y_sol = round(new_y11[idx1[0]][0].astype(float),2)
            x_sol = round(new_x[idx1[0]][0].astype(float),2)
            y2 = res_df['VISCOSITY'].values
            z = np.poly1d(np.polyfit(x, y2, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y12 = interp1(new_x)
            new_y22 = interp2(new_x)
            idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
            y_sol = round(new_y12[idx2[0]][0].astype(float),2)
            x_sol = round(new_x[idx2[0]][0].astype(float),2)
            y_list3.append(bat_2(lx, dP, kh, new_y12[idx2[0]], kh*kk[k], rw, h, L, new_y11[idx1[0]])[0])
        df_y.append(y_list3)
        for i in df_time.values:
            dP = i[1] - i[2]
            eclfiles = EclFiles(f'model_folder/{filename}')
            dframe = pvt.df(eclfiles)
            indic = dframe['KEYWORD'] == 'PVDO'
            res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
            x = res_df['PRESSURE'].values
            y1 = res_df['VOLUMEFACTOR'].values
            z = np.poly1d(np.polyfit(x, y1, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y11 = interp1(new_x)
            new_y21 = interp2(new_x)
            idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
            y_sol = round(new_y11[idx1[0]][0].astype(float),2)
            x_sol = round(new_x[idx1[0]][0].astype(float),2)
            y2 = res_df['VISCOSITY'].values
            z = np.poly1d(np.polyfit(x, y2, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y12 = interp1(new_x)
            new_y22 = interp2(new_x)
            idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
            y_sol = round(new_y12[idx2[0]][0].astype(float),2)
            x_sol = round(new_x[idx2[0]][0].astype(float),2)
            y_list4.append(bat_3(S, lx, ly, L, kh, kh*kk[k], rw, h, dP, new_y12[idx2[0]], new_y11[idx1[0]])[0])
        df_y.append(y_list4)
        for i in df_time.values:
            dP = i[1] - i[2]
            eclfiles = EclFiles(f'model_folder/{filename}')
            dframe = pvt.df(eclfiles)
            indic = dframe['KEYWORD'] == 'PVDO'
            res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
            x = res_df['PRESSURE'].values
            y1 = res_df['VOLUMEFACTOR'].values
            z = np.poly1d(np.polyfit(x, y1, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y11 = interp1(new_x)
            new_y21 = interp2(new_x)
            idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
            y_sol = round(new_y11[idx1[0]][0].astype(float),2)
            x_sol = round(new_x[idx1[0]][0].astype(float),2)
            y2 = res_df['VISCOSITY'].values
            z = np.poly1d(np.polyfit(x, y2, 2))
            xp = np.linspace(0, 600, 100)
            sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
            sol_list_y = [0, 1, 2, 3]
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
            interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
            new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
            new_y12 = interp1(new_x)
            new_y22 = interp2(new_x)
            idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
            y_sol = round(new_y12[idx2[0]][0].astype(float),2)
            x_sol = round(new_x[idx2[0]][0].astype(float),2)
            y_list5.append(bat_1(lx, dP, kh, new_y12[idx2[0]][0], kh*kk[k], rw, h, L, new_y11[idx1[0]][0]))
        df_y.append(y_list5)
        for i in range(0, 5):
            fig.add_trace(go.Scatter(
                        x=df_time.index,
                        y=df_y[i+4],
                        mode='lines',
                        name=f'{trace_name[i]}; аниз. - '+str(kk[k])))
        fig.update_xaxes(tickformat='%d.%m.%y')
        fig.update_layout(title=go.layout.Title(text=title),
                               xaxis_title='Дата',
                               yaxis_title='Qж, м3/сут')
    return [fig, df_y]

In [36]:
rw_ = rw[0] # радиус скважины, м
kh = permx_list[0] # горизонтальная проницаемость, мд
h = nz*dz # мощность пласта, м
L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
re = lx/2 # радиус контура питания, м
figure = figure_plot(rw_, kh, h, kk, L, zw, re, lx, ly, skin[0], start_date, model_list[0])[0]
df_y = figure_plot(rw_, kh, h, kk, L, zw, re, lx, ly, skin[0], start_date, model_list[0])[1]
iplot(figure)
print(df_y)

[[Timestamp('2020-01-02 00:00:00'), Timestamp('2020-01-05 00:00:00'), Timestamp('2020-01-13 13:03:55'), Timestamp('2020-02-01 17:48:38'), Timestamp('2020-03-01 00:00:00'), Timestamp('2020-04-30 00:00:00'), Timestamp('2020-06-29 00:00:00'), Timestamp('2020-08-28 00:00:00'), Timestamp('2020-10-27 00:00:00')], [131.44386291503906, 92.9864273071289, 68.52416229248047, 54.20751953125, 47.13592529296875, 41.8604850769043, 39.51884460449219, 38.36334609985352, 37.76805877685547], [1.2506474083813313, 1.2506474083813313, 1.2506474083813313, 1.2506474083813313, 1.2506474083813313, 1.2506474083813313, 1.2506474083813313, 1.2506474083813313, 1.2506474083813313], [1.2464389464404666, 1.2464389464404666, 1.2464389464404666, 1.2464389464404666, 1.2464389464404666, 1.2464389464404666, 1.2464389464404666, 1.2464389464404666, 1.2464389464404666], [48.14194025850309, 48.14170206034614, 48.14105552249159, 48.1399666166313, 48.13855444184372, 48.13573009226858, 48.13295678515564, 48.1302345205049, 48.1275

In [53]:
import pandas as pd
import os
import xlsxwriter
import datetime

XL_PATH = 'ResPlot1.xlsx'
workbook = xlsxwriter.Workbook(XL_PATH)
worksheet = workbook.add_worksheet()    
date_format = workbook.add_format({'num_format': 'd mmmm yyyy'})
worksheet.write('L2', 'VOLUMEFACTOR')
worksheet.write('M2', 'VISCOSITY')

i = 3
j = 0
for data in df_y[0]:
    worksheet.write_datetime(f'E{i}', data, date_format)
    worksheet.write(f'F{i}', df_y[1][j])
    worksheet.write(f'L{i}', df_y[2][j])
    worksheet.write(f'M{i}', df_y[3][j])
    worksheet.write(f'G{i}', df_y[4][j])
    worksheet.write(f'H{i}', df_y[5][j])
    worksheet.write(f'I{i}', df_y[6][j])
    worksheet.write(f'J{i}', df_y[7][j])
    worksheet.write(f'K{i}', df_y[8][j])
    i += 1
    j += 1
    
workbook.close()

In [ ]:
rw_ = rw[0] # радиус скважины, м
kh = permx_list[1] # горизонтальная проницаемость, мд
h = nz*dz # мощность пласта, м
L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
re = lx/2 # радиус контура питания, м
figure = figure_plot(filename, rw_, kh, h, kk, L, zw, re, lx, ly, skin[0], start_date, model_list[1])

iplot(figure)

In [ ]:
for i in range(0, 4):
    filename = f'TEST_MODEL_HORIZONTAL.{i}2'
    rw_ = rw[0] # радиус скважины, м
    kh = permx_list[2] # горизонтальная проницаемость, мд
    h = nz*dz # мощность пласта, м
    L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
    zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
    re = lx/2 # радиус контура питания, м
    figure = figure_plot(filename, rw_, kh, h, kk[i], L, zw, re, lx, ly, skin[0], start_date, model_list[2])

    iplot(figure)

In [ ]:
title = 'Динамика пластового давления по годам'
x_axis = 'Дата'
y_axis = 'Pпл, атм'
model.summ_plot(['FPR'], x_axis=x_axis, y_axis=y_axis, title=title, name=model_list)

In [ ]:
filename = 'TEST_MODEL_HORIZONTAL.00'
parametr = 'PRESSURE' # 'DX', 'SOIL', 'PORO'
model.grid_plot(filename, parametr) 